# README.

This notebook is the entrypoint for Azure ML enabled training.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the training.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for training.

# Imports.

In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import shutil
from pathlib import Path

import glob2 as glob
from azureml.core import Dataset, Experiment, Environment, Run, ScriptRunConfig, Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import MpiConfiguration

from src.constants import REPO_DIR
from src.train_util import copy_dir
from src.config import CONFIG, DATASET_MODE_MOUNT, DATASET_MODE_DOWNLOAD

# Setting screws

In [ ]:
dataset_name = "anon-depthmap-95k"  # "anon-depthmap-mini"  # "anon-depthmap-95k"  # anon-depthmaps-4k  # anon-depthmaps-56k
experiment_name = "q3-depthmapmultiartifactlatefusion-plaincnn-height-95k"
tags = {}

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [ ]:
code_dir = Path("src")
temp_path = Path("temp_train")
copy_dir(src=code_dir, tgt=temp_path, glob_pattern='*.py')
copy_dir(src=REPO_DIR / "src/common", tgt=temp_path / "temp_common", glob_pattern='*/*.py', should_touch_init=True)

# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [ ]:
workspace = Workspace.from_config()
workspace

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.

In [ ]:
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [ ]:
cluster_name = "gpu-cluster"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

# Get the dataset for training.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

In [ ]:
dataset = workspace.datasets[dataset_name]
dataset

# Push the training source code to Azure.

Creates an estimator (a template for a compute cluster node) and pushes it to the compute cluster.

In [ ]:
script_params = {f"--{k}": v for k, v in CONFIG.items()}
script_params

In [ ]:
curated_env_name = "cgm-env-v15-opencv"

ENV_EXISTS = True
if ENV_EXISTS:
    cgm_env = Environment.get(workspace=workspace, name=curated_env_name)
else:
    cgm_env = Environment.from_conda_specification(name=curated_env_name, file_path=REPO_DIR / "environment_train.yml")
    cgm_env.docker.enabled = True
    cgm_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
    # cgm_env.register(workspace)  # Please be careful not to overwrite existing environments

In [ ]:
if CONFIG.DATASET_MODE == DATASET_MODE_MOUNT:
    dataset_argument = dataset.as_named_input('cgm_dataset').as_mount()
elif CONFIG.DATASET_MODE == DATASET_MODE_DOWNLOAD:
    dataset_argument = dataset.as_named_input('cgm_dataset').as_download()
else:
    raise Exception("Please specify DATASET_MODE")

In [ ]:
# Create the ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory=temp_path,
                                    compute_target=compute_target,
                                    script='train.py',
                                    arguments=[dataset_argument] + [str(item) for sublist in script_params.items() for item in sublist],
                                    environment=cgm_env,
)

# Set compute target.
script_run_config.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(config=script_run_config, tags=tags)

# Show run.
run

# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [ ]:
shutil.rmtree(temp_path)